<a href="https://colab.research.google.com/github/abhranil-datascience/ClassificationPOC/blob/master/KNNClassificationPOC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
############################## Mount Drive ######################################## 
from google.colab import drive
drive.mount('/content/gdrive')
############################## Change Directory ###################################
import os
os.chdir('/content/gdrive/My Drive/MLandDLFullCourse/ML/Classification/KNearestNeighbors')
########## Suppress Warnings ############
import warnings
warnings.filterwarnings('ignore')
########### Import Dataset #############
import pandas as pd
dataset=pd.read_csv('Social_Network_Ads.csv')
X=dataset.iloc[:,1:4].values
Y=dataset.iloc[:,4].values

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
########### Categorical Variable ###############
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
X[:,0]=le.fit_transform(X[:,0])
########## Feature Scaling ##############
from sklearn.preprocessing import MinMaxScaler
SC=MinMaxScaler(feature_range=(0,1))
XScaled=SC.fit_transform(X)
########## Train Test Split #############
from sklearn.model_selection import train_test_split
XTrain,XTest,YTrain,YTest=train_test_split(XScaled,Y,test_size=0.1,random_state=42)
########## Check VIF ####################
from statsmodels.stats.outliers_influence import variance_inflation_factor
VIF=[0]*XTrain.shape[1]
for col in range(0,XTrain.shape[1]):
  VIF[col]=VIF[col]+variance_inflation_factor(exog=XTrain,exog_idx=col)#1.6142518559270171, 2.9082305467018816, 2.753146445004674
VIF

[1.6142518559270171, 2.9082305467018816, 2.753146445004674]

In [0]:
########## GridSearchCV hyperparameter tuning #################
from sklearn.neighbors import KNeighborsClassifier
RegressorKNN_GS=KNeighborsClassifier()
params_GS=[{'n_neighbors':[3,5,10,15,20,25,30],
           'weights':['uniform','distance'],
           'algorithm':['auto'],
           'leaf_size':[20,25,30,35,40,45,50],
           'p':[1,2],
           'metric':['euclidean','manhattan','chebyshev','minkowski',]}]
from sklearn.model_selection import GridSearchCV
GS=GridSearchCV(estimator=RegressorKNN_GS,param_grid=params_GS,scoring='accuracy',cv=10)
GS.fit(XTrain,YTrain)
best_params_KNN=GS.best_params_
best_params_KNN

{'algorithm': 'auto',
 'leaf_size': 20,
 'metric': 'manhattan',
 'n_neighbors': 15,
 'p': 1,
 'weights': 'uniform'}

In [0]:
######### KFoldCrossValidation #################
RegressorKNN=KNeighborsClassifier(algorithm='auto',leaf_size=20,metric='manhattan',n_neighbors=15,p=1,weights='uniform')
from sklearn.model_selection import KFold
from sklearn import metrics as mt
import statistics as st
Accuracy=[]
kf=KFold(n_splits=40,shuffle=True)
for count in range(0,100):
  for train_index,val_index in kf.split(XTrain):
    XTrainKF,YTrainKF=XTrain[train_index],YTrain[train_index]
    XValKF,YValKF=XTrain[val_index],YTrain[val_index]
    RegressorKNN.fit(XTrainKF,YTrainKF)
    YPredKF=RegressorKNN.predict(XValKF)
    Accuracy.append(mt.accuracy_score(YValKF,YPredKF))
print("Average Accuracy: "+str(st.mean(Accuracy)))#0.9086388888888889

Average Accuracy: 0.9086666666666666


In [0]:
RegressorKNN.fit(XTrain,YTrain)
YPred=RegressorKNN.predict(XTest)
cm=mt.confusion_matrix(YTest,YPred)
cm

array([[25,  2],
       [ 0, 13]])